In [1]:
import json
from pathlib import Path
import sys
sys.path.append("../") 
from src.models.ir.baseline import BaselineIR
import html
from bs4 import BeautifulSoup
import html2text

h = html2text.HTML2Text()
h.ignore_links = True
h.ignore_pictures = True

ModuleNotFoundError: No module named 'src.models.ir.baseline'

In [2]:
p = Path("../data/raw/task1/crag_task_1_v2.jsonl")

In [49]:
ir_model = BaselineIR(max_sentences=10)

In [66]:
def string_cleaner(string):
    # Unescape HTML entities
    string = html.unescape(string)
    
    # Replace inconsistent quotes and remove unnecessary backslashes
    string = string.replace('\'s', '’s')  # use typographic apostrophe
    string = string.replace('“', '"').replace('”', '"')  # standardize to double quotes
    string = string.replace('\\', '')  # remove unnecessary backslashes
    return string

def string_cleaner2(string):
    soup = BeautifulSoup(string, 'html.parser')
    text = soup.get_text()
    return text


def string_cleaner3(string):
    soup = BeautifulSoup(string, 'html.parser')
    paragraphs = soup.find_all('p')
    return text


def formatting_fn_v0(html_dict, title):
    segments = []
    for key in html_dict:
        segments.append(key)
        segments.extend(html_dict[key])
    return segments


def formatting_fn_v1(html_dict, title):
    segments = []
    for key in html_dict:
        for paragraph in html_dict[key]:
            formatted_text = f"{key}:\n{paragraph}"
            segments.append(formatted_text)
    return segments


def formatting_fn_v2(html_dict, title):
    segments = []
    for key in html_dict:
        for paragraph in html_dict[key]:
            formatted_text = f"{title}\n=====\n{key}:\n{paragraph}"
            segments.append(formatted_text)
    return segments


def extract_text_by_headers_html(page_html, page_name, version="v2"):
    formatting_fns = {
        "v0":formatting_fn_v0,
        "v1":formatting_fn_v1,
        "v2":formatting_fn_v2,
    }
    formatting_fn = formatting_fns[version]
    soup = BeautifulSoup(page_html, 'html.parser')
    content = {}
    current_header = None

    # Iterate over all elements that are headers or contain text
    for element in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'div', 'li']):
        if element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            # Start a new header section
            current_header = element.get_text().strip()
            content[current_header] = []
        else:
            # Add text to the current header section
            if current_header:
                text = element.get_text(separator=' ', strip=True)
                if text:
                    content[current_header].append(text)
            else:
                # Handle case where text appears before the first header
                if 'No Header' not in content:
                    content['No Header'] = []
                text = element.get_text(separator=' ', strip=True)
                if text:
                    content['No Header'].append(text)

    return formatting_fn(content, page_name)

In [71]:
n = -1
json_lines = []
with open(p) as f:
    for line in f:
        if len(json_lines) == n:
            break
        json_lines.append(json.loads(line))

In [72]:
len(json_lines)

2735

In [68]:
def print_info(json_obj):
    query = json_obj["query"]
    print(json_obj["query"])
    print(json_obj["answer"])
    filtered_results = {}
    for result in json_obj["search_results"]:
        filtered_results[result["page_name"]] = result
    filtered_results = list(filtered_results.values())
    # document_strings = [string_cleaner(result["page_snippet"]) for result in json_obj["search_results"]]
    # document_strings = [string_cleaner2(result["page_result"]) for result in json_obj["search_results"]]
    document_strings = []
    document_strings.extend(extract_text_by_headers_html(result["page_result"], result["page_name"]))
    document_strings = list(set(document_strings))
    top_sentences = ir_model.get_top_sentences(query, document_strings)
    print("="*100)
    # print(json_obj["answer"])
    # print(len(top_sentences))
    for sentence, score in top_sentences:
        print(f"{score}\n{sentence}")
        print("-"*100)

In [70]:
print_info(json_lines[5])

which movie won the academy award for best picture in 2018, categorized under the fantasy genre?
the shape of water
8.408863067626953
Academy Award for Best Picture - Wikipedia
=====
Genres[edit]:
Only two fantasy films have won: The Lord of the Rings: The Return of the King (2003) and The Shape of Water (2017), although more have been nominated.
----------------------------------------------------------------------------------------------------
7.064507484436035
Academy Award for Best Picture - Wikipedia
=====
Genres[edit]:
No comic book film has won, and only three have ever been nominated: Skippy (1931), Black Panther (2018), and Joker (2019). [27]
----------------------------------------------------------------------------------------------------
6.009922504425049
Academy Award for Best Picture - Wikipedia
=====
Genres[edit]:
The Silence of the Lambs (1991) is the only horror film to win Best Picture, and only five others have been nominated for Best Picture: The Exorcist (1973), J

In [16]:
for result in json_lines[55]["search_results"]:
    print(result["page_name"], result["page_url"])
    print(result)

Black Death - Wikipedia https://en.wikipedia.org/wiki/Black_Death
{'page_name': 'Black Death - Wikipedia', 'page_url': 'https://en.wikipedia.org/wiki/Black_Death', 'page_snippet': 'Nestorian gravesites dating from 1338 to 1339 near Issyk-Kul have inscriptions referring to plague, which has led some historians and epidemiologists to think they mark the outbreak of the epidemic; this is supported by recent direct findings of Y. pestis DNA in teeth samples from graves in ...Nestorian gravesites dating from 1338 to 1339 near Issyk-Kul have inscriptions referring to plague, which has led some historians and epidemiologists to think they mark the outbreak of the epidemic; this is supported by recent direct findings of Y. pestis DNA in teeth samples from graves in the area with inscriptions referring to "pestilence" as the cause of death. Epidemics killed an estimated 25 million across Asia during the fifteen years before the Black Death reached Constantinople in 1347. The evidence does not s

In [ ]:
ir_model.get_sentences(string_cleaner(json_lines[0]["search_results"][0]["page_snippet"]))

/home/mitchell/software/miniconda3/envs/meta_rag/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


['A disturbing peek into the world of a reckless and selfish father posing as the ultimate, open-minded, pill-popping dude, the subject of the docu "Small Town Ecstasy’s" makes the skin crawl.',
 'Scott’s a 40-year-old preacher’s son and a raver, enamored by marijuana and the synthetic drug Ecstasy, ...',
 'A disturbing peek into the world of a reckless and selfish father posing as the ultimate, open-minded, pill-popping dude, the subject of the docu "Small Town Ecstasy’s" makes the skin crawl.',
 'Scott’s a 40-year-old preacher’s son and a raver, enamored by marijuana and the synthetic drug Ecstasy, who puts his children’s future at risk through his lassez faire approach to child-rearing.',
 'Like guests willing to go on talkshows to learn a "secret," it’s almost inexplicable why this family would want the cameras on them.',
 'It’s a tribute to the filmmakers that they could get the family to share intimate moments — Craig smoking dope rather than attending one of his father’s hearing

In [ ]:
for obj in json_lines:
    print(obj["query"] + "   " + obj["answer"] + "   " + obj["question_type"])

which one of these came out earlier, the greater meaning of water or small town ecstasy?   false   comparison
who are the original bad boy pistons?   isiah thomas, joe dumars and bill laimbeer.   set
which film is older, superbad or blue crush?   blue crush (2002)   comparison
what was the date of the first dividend payout for gold?   1987-11-23   simple
what is the name of the #1 single that shakira released in 2013?   shakira did not release a single in 2013.   false_premise
why does a butterfly live longer than 2 years?   most butterfly species only live one to two weeks, not 2 years or more.   false_premise
who was the first actor to play the role of batman in a live-action movie?   lewis wilson was the first actor to play the role of batman in a live-action movie, in the 1943 film the batman.   multi-hop
what is plato's theory of forms?   plato's theory of forms suggests that the physical world is not as real or true as "forms". according to this theory, forms—conventionally capit

In [ ]:
obj.keys()

dict_keys(['interaction_id', 'query_time', 'domain', 'question_type', 'static_or_dynamic', 'query', 'answer', 'search_results', 'split', 'alternative_answers'])